To classify images into these 10 superclasses and 100 classes, we'll utilize a convolutional neural network (CNN) architecture, one of the most commonly used architectures for image classification tasks. Here's a basic outline for the process:

# 1. Data Preprocessing:
Loading Data: Use a suitable data loader to read images.
Data Augmentation: Apply data augmentation techniques like rotation, scaling, horizontal flip, etc., to increase the dataset's diversity.
Splitting the Data: Partition the dataset into training, validation, and test sets.

# 2. Building the CNN Model:
Here's a basic CNN architecture to start with:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)), # Assuming images are 32x32x3
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='softmax') # 100 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 3. Training the Model:
Once you have the model ready, you can train it using the training data.

In [ ]:
history = model.fit(training_data, training_labels, epochs=50, validation_data=(validation_data, validation_labels))

# 4. Model Evaluation:
After training, evaluate the model's performance on the test dataset.

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print("Test accuracy: ", test_acc)

# 5. Fine-tuning and Optimization:
Transfer Learning: If training from scratch doesn't give good results, consider using transfer learning. Pre-trained models like VGG16, ResNet, etc., can be utilized. These models are trained on large datasets and can help in achieving better accuracy.

Hyperparameter Tuning: We can use tools like Keras Tuner or techniques like random search and grid search to find the best hyperparameters for the model.

# 6. Deployment:
Once satisfied with the model's performance, you can deploy it for real-time predictions.

# 7. Notes:
Dataset Size: The architecture might need adjustments depending on the dataset's size. If the dataset is too small, consider using techniques like transfer learning.

Image Size: The example assumes images of size 32x32x3. If the actual size is different, adjust the input_shape accordingly.

Label Encoding: Make sure to one-hot encode the labels before training as the model uses 'categorical_crossentropy' as the loss function.